# Data Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Insert, change the directory 
import sys
sys.path.insert(0,'/content/drive/MyDrive/graph-based-fraud-detection-main')
%cd /content/drive/MyDrive/graph-based-fraud-detection-main

/content/drive/MyDrive/graph-based-fraud-detection-main


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# pd.read_csv('./ieee-data/test_transaction.csv')

In [ ]:
transaction_df = pd.read_csv('./ieee-data/train_transaction.csv')
identity_df = pd.read_csv('./ieee-data/train_identity.csv')
# test_transaction = pd.read_csv('./ieee-data/test_transaction.csv')
# test_identity = pd.read_csv('./ieee-data/test_identity.csv')
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 394 entries, TransactionID to V339
dtypes: float64(384), int64(4), object(6)
memory usage: 150.3+ MB


In [ ]:
identity_df

In [ ]:
transaction_df

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3376672,0,9766193,34.742,C,11201,103.0,185.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3535390,0,14487694,22.993,C,1164,545.0,185.0,visa,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3041260,0,1275065,100.000,R,5409,170.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3492547,0,13228103,15.985,C,4461,375.0,185.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3310396,0,8015461,20.000,S,8937,399.0,150.0,american express,150.0,...,0.0,0.0,140.0,0.0,0.0,90.0,0.0,0.0,50.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,3316436,1,8107600,36.856,C,14276,177.0,185.0,mastercard,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49996,3391195,1,10196368,67.067,C,9642,103.0,185.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49997,3395166,1,10308769,109.965,C,11805,545.0,185.0,visa,137.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49998,3369564,1,9580785,13.655,C,9633,130.0,185.0,visa,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
id_cols = ['card1','card2','card3','card4','card5','card6','ProductCD','addr1','addr2','P_emaildomain','R_emaildomain']
cat_cols = ['M1','M2','M3','M4','M5','M6','M7','M8','M9']
train_data_ratio = 0.8

In [ ]:
#shuffle the dataframe if required
transaction_df = transaction_df.sample(frac=1)

In [ ]:
transaction_df.shape

(50000, 394)

Based on the train/test ratio we assigned before, extact the IDs of test data. 

In [ ]:
n_train = int(transaction_df.shape[0]*train_data_ratio)
test_ids = transaction_df.TransactionID.values[n_train:]

In [ ]:
get_fraud_frac = lambda series: 100 * sum(series)/len(series)
print("Percent fraud for train transactions: {}".format(get_fraud_frac(transaction_df.isFraud[:n_train])))
print("Percent fraud for test transactions: {}".format(get_fraud_frac(transaction_df.isFraud[n_train:])))
print("Percent fraud for all transactions: {}".format(get_fraud_frac(transaction_df.isFraud)))

Percent fraud for train transactions: 3.0
Percent fraud for test transactions: 3.0
Percent fraud for all transactions: 3.0


 Save test IDs into the `test.csv` file

In [ ]:
with open('data/test.csv', 'w') as f:
    f.writelines(map(lambda x: str(x) + "\n", test_ids))

Based on the standard we talked about before, define non-feature-columns and feature-columns for creating graph.

In [ ]:
non_feature_cols = ['isFraud', 'TransactionDT'] + id_cols
print(non_feature_cols)

['isFraud', 'TransactionDT', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain']


In [ ]:
feature_cols = [col for col in transaction_df.columns if col not in non_feature_cols]
print(feature_cols)

['TransactionID', 'TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102',

Transfer categorical features to be dummy variables and scale the `TransactionAmt` feature by log10.

In [ ]:
features = pd.get_dummies(transaction_df[feature_cols], columns=cat_cols).fillna(0)
features['TransactionAmt'] = features['TransactionAmt'].apply(np.log10)

In [ ]:
features.shape
# their result - (590540, 391)
# mine - (590540, 391)

(50000, 375)

In [ ]:
print(list(features.columns))

['TransactionID', 'TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109

Save the features into `features.csv` for future training. 

p.s. We don't need the index column and header.

In [ ]:
features.to_csv('data/features.csv', index=False, header=False)

Save the IDs and label into the `tags.csv`.

In [ ]:
transaction_df[['TransactionID', 'isFraud']].to_csv('data/tags.csv', index=False)

Select the columns that define the edges.

In [ ]:
edge_types = id_cols + list(identity_df.columns)
print(edge_types)

['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']


In [ ]:
all_id_cols = ['TransactionID'] + id_cols
full_identity_df = transaction_df[all_id_cols].merge(identity_df, on='TransactionID', how='left')
full_identity_df.head(5)

,TransactionID,card1,card2,card3,card4,card5,card6,ProductCD,addr1,addr2,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3090845,18132,567.0,150.0,mastercard,117.0,debit,R,204.0,87.0,...,mobile safari 11.0,32.0,2436x1125,match_status:1,T,F,F,F,mobile,iOS Device
1,2999937,3154,408.0,185.0,mastercard,224.0,debit,C,NaN,NaN,...,chrome 62.0 for android,NaN,NaN,NaN,F,F,T,T,mobile,SM-G925I Build/LMY47X
2,3272490,6108,378.0,146.0,mastercard,219.0,debit,C,NaN,NaN,...,chrome 63.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
3,3069882,3148,111.0,150.0,visa,226.0,debit,S,476.0,87.0,...,ie 11.0 for desktop,24.0,1280x1024,match_status:2,T,T,T,T,desktop,Trident/7.0
4,3266459,10023,111.0,150.0,visa,226.0,debit,H,441.0,87.0,...,mobile safari generic,32.0,2436x1125,match_status:1,T,F,F,T,mobile,iOS Device


For each identity feature, save the data into the corresponding `relation_{FEATURE NAME}_edgelist.csv`. Each csv file represents one kind of edge.

In [ ]:
edges = {}
for etype in edge_types:
    edgelist = full_identity_df[['TransactionID', etype]].dropna()
    edgelist.to_csv('data/relation_{}_edgelist.csv'.format(etype), index=False, header=True)
    edges[etype] = edgelist

print(edges)

{'card1':        TransactionID  card1
0            3090845  18132
1            2999937   3154
2            3272490   6108
3            3069882   3148
4            3266459  10023
...              ...    ...
49995        3159000   2650
49996        3039694  14749
49997        3479103  14128
49998        3438307   3154
49999        3082481   2755

[50000 rows x 2 columns], 'card2':        TransactionID  card2
0            3090845  567.0
1            2999937  408.0
2            3272490  378.0
3            3069882  111.0
4            3266459  111.0
...              ...    ...
49995        3159000  142.0
49996        3039694  555.0
49997        3479103  170.0
49998        3438307  408.0
49999        3082481  404.0

[49728 rows x 2 columns], 'card3':        TransactionID  card3
0            3090845  150.0
1            2999937  185.0
2            3272490  146.0
3            3069882  150.0
4            3266459  150.0
...              ...    ...
49995        3159000  185.0
49996        3039694  

Let's re-check the edges we defined.

In [ ]:
import glob

file_list = glob.glob('./data/*edgelist.csv')

edges = ",".join(map(lambda x: x.split("/")[-1], [file for file in file_list if "relation" in file]))

edges_full = ''
for etype in edge_types:
    edges_full += ',data/relation_{}_edgelist.csv'.format(etype)


In [ ]:
edges

'relation_card1_edgelist.csv,relation_card2_edgelist.csv,relation_card3_edgelist.csv,relation_card4_edgelist.csv,relation_card5_edgelist.csv,relation_card6_edgelist.csv,relation_ProductCD_edgelist.csv,relation_addr1_edgelist.csv,relation_addr2_edgelist.csv,relation_P_emaildomain_edgelist.csv,relation_R_emaildomain_edgelist.csv,relation_TransactionID_edgelist.csv,relation_id_01_edgelist.csv,relation_id_02_edgelist.csv,relation_id_03_edgelist.csv,relation_id_04_edgelist.csv,relation_id_05_edgelist.csv,relation_id_06_edgelist.csv,relation_id_07_edgelist.csv,relation_id_08_edgelist.csv,relation_id_09_edgelist.csv,relation_id_10_edgelist.csv,relation_id_11_edgelist.csv,relation_id_12_edgelist.csv,relation_id_13_edgelist.csv,relation_id_14_edgelist.csv,relation_id_15_edgelist.csv,relation_id_16_edgelist.csv,relation_id_17_edgelist.csv,relation_id_18_edgelist.csv,relation_id_19_edgelist.csv,relation_id_20_edgelist.csv,relation_id_21_edgelist.csv,relation_id_22_edgelist.csv,relation_id_23_edge